In [75]:
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import cv2
import fnmatch
import os
import glob

In [76]:
df = pd.read_csv('data.txt', sep=" ", header=None)
df_normal_marked = pd.read_csv('saved.csv')
df.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]
df.index = df.index + 1
df = df.dropna()

In [77]:
df_final = pd.concat([df, df_normal_marked], axis=0)

In [79]:
df_final = df_final.reset_index(drop=True)
del df_final["Unnamed: 0"]

In [80]:
df_final

,ID,background_tissue,class,severity,x,y,rad
0,mdb001,G,CIRC,B,535,425,197.0
1,mdb002,G,CIRC,B,522,280,69.0
2,mdb005,F,CIRC,B,477,133,30.0
3,mdb005,F,CIRC,B,500,168,26.0
4,mdb010,F,CIRC,B,525,425,33.0
...,...,...,...,...,...,...,...
307,mdb318,D,NORM,NaN,442,68,50.0
308,mdb319,D,NORM,NaN,590,281,50.0
309,mdb320,D,NORM,NaN,495,264,50.0
310,mdb321,D,NORM,NaN,604,271,50.0


In [3]:
df_normal = pd.read_csv('data.txt', sep=" ", header=None)
df_normal.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]

In [4]:
df_normal = df_normal.loc[df_normal['class']  == 'NORM']
df_normal = df_normal.reset_index(drop=True)

In [26]:
df_normal

,ID,background_tissue,class,severity,x,y,rad
0,mdb003,D,NORM,NaN,NaN,NaN,NaN
1,mdb004,D,NORM,NaN,NaN,NaN,NaN
2,mdb006,F,NORM,NaN,NaN,NaN,NaN
3,mdb007,G,NORM,NaN,NaN,NaN,NaN
4,mdb008,G,NORM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
202,mdb318,D,NORM,NaN,NaN,NaN,NaN
203,mdb319,D,NORM,NaN,NaN,NaN,NaN
204,mdb320,D,NORM,NaN,NaN,NaN,NaN
205,mdb321,D,NORM,NaN,NaN,NaN,NaN


In [ ]:
df['severity'].value_counts()

In [ ]:
im = Image.open('all-mias/mdb001.pgm') # PILLOW image processing

In [ ]:
im.getpixel( (535,425) )

In [ ]:
last = 'start'

In [ ]:
def img_draw(image, x, y, r, c):
    if c=='M':
        color = 'red'
    else:
        color = 'blue'
    draw = ImageDraw.Draw(image)
    leftUpPoint = (x-r, y-r)
    rightDownPoint = (x+r, y+r)
    twoPointList = [leftUpPoint, rightDownPoint]
    draw.ellipse(twoPointList, outline=color, width=5)

In [ ]:
for index, row in df.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    if last != path:
        im = Image.open(path).convert("RGB")
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    c = row['severity']
    img_draw(im, x, 1024 - y, r, c)
    im.save("marked/{}.png".format(row['ID']))
    last = path

In [95]:
mean = []
var = []
std = []
max_v = []
min_v = []
min_coor_x = []
min_coor_y = []
max_coor_x = []
max_coor_y = []
for index, row in df_final.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    im = cv2.imread('all-mias/{name}.pgm'.format(name=row['ID']))
    h,w,d = im.shape
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    #c = row['severity']
    mask = np.zeros((h,w), np.uint8)
    cv2.circle(mask,(x,1024-y),r ,1,thickness=-1)
    masked_data = cv2.bitwise_and(im, im, mask=mask)
    mean.append(cv2.meanStdDev(im, mask = mask)[0][0])
    var.append(np.power(cv2.meanStdDev(im, mask = mask)[1][0], 2))
    std.append(cv2.meanStdDev(im, mask = mask)[1][0])
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(gray, mask = mask)
    
    max_v.append(max_val)
    min_v.append(min_val)
    min_coor_x.append(min_loc[0])
    min_coor_y.append(min_loc[1])
    max_coor_x.append(max_loc[0])
    max_coor_y.append(max_loc[1])

In [96]:
df_final['mean'] = np.array(mean)
df_final['var'] = np.array(var)
df_final['std'] = np.array(std)
df_final['min_pixel'] = np.array(min_v)
df_final['max_pixel'] = np.array(max_v)

df_final['min_coor_x'] = np.array(min_coor_x)
df_final['min_coor_y'] = np.array(min_coor_y)
df_final['max_coor_x'] = np.array(max_coor_x)
df_final['max_coor_y'] = np.array(max_coor_y)

In [97]:
df_final

,ID,background_tissue,class,severity,x,y,rad,mean,var,std,min_pixel,max_pixel,min_coor_x,min_coor_y,max_coor_x,max_coor_y
0,mdb001,G,CIRC,B,535,425,197.0,150.781051,3205.504934,56.617179,5.0,221.0,373,710,487,632
1,mdb002,G,CIRC,B,522,280,69.0,197.870827,764.537398,27.650269,75.0,229.0,571,792,531,679
2,mdb005,F,CIRC,B,477,133,30.0,149.605105,409.222292,20.229243,84.0,187.0,481,920,476,876
3,mdb005,F,CIRC,B,500,168,26.0,162.426214,80.178549,8.954248,128.0,181.0,520,841,522,859
4,mdb010,F,CIRC,B,525,425,33.0,184.385157,127.091607,11.273491,155.0,208.0,557,599,519,598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,mdb318,D,NORM,NaN,442,68,50.0,14.033015,0.505857,0.711236,11.0,35.0,436,951,467,965
308,mdb319,D,NORM,NaN,590,281,50.0,145.912938,28.130725,5.303841,130.0,168.0,587,778,637,745
309,mdb320,D,NORM,NaN,495,264,50.0,195.578840,470.810515,21.698168,146.0,227.0,451,745,527,722
310,mdb321,D,NORM,NaN,604,271,50.0,177.764946,30.464061,5.519426,163.0,199.0,626,740,555,746
